In [113]:
# Load library
import csv
import pandas as pd
import numpy as np
import time

In [106]:
data = pd.read_csv("data/international-football-results-from-1872-to-2017.csv")
data.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland
1,1873-03-08,England,Scotland,4,2,Friendly,London,England
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland
3,1875-03-06,England,Scotland,2,2,Friendly,London,England
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland


In [125]:
# Put everything together
# 1. Read data:

# 2. Add new column named "id" and "winner"
data['id'] = data.index
data['winner'] = np.where(data.home_score > data.away_score, data.home_team, 
                                     np.where(data.home_score < data.away_score, data.away_team, 'None'))
# filter data from 2005
data_2005 = data[data['date'] > '2005-01-01'].copy()
# data_2005 = data[data['date'] > '2016-01-01'].copy()
# define my own dataframe
col_names = ['id','date', 'team_1', 'team_2', 'home_team', 'tournament', 'h_win_diff', 'h_draw', 'f_goalF_1', 
             'f_goalF_2','f_goalA_1','f_goalA_2', 'f_win_1', 'f_win_2', 'f_draw_1', 'f_draw_2','ratio_odds','avg_odds_draw','result']
my_df = pd.DataFrame(columns = col_names)

start_time = time.time()
data_len = data_2005.shape[0]
for i in range(data_len):
#     print(i)
    # Current Match
    curr_match = data_2005.iloc[i,:]
    
    # Get the head-to-head result  
    pre_df = curr_match[['id','date','tournament']].copy() #Initialize dataframe
    pre_df['team_1'] =  curr_match['home_team'] if curr_match['home_team'] < curr_match['away_team'] else curr_match['away_team'] 
    pre_df['team_2'] =  curr_match['away_team'] if curr_match['home_team'] < curr_match['away_team'] else curr_match['home_team'] 
    pre_df['home_team'] = curr_match['home_team']
    
    team_1 = pre_df['team_1']
    team_2 = pre_df['team_2']
    
    df_h2h = data[(((data.home_team == team_1) & (data.away_team == team_2)) | 
                  ((data.home_team == team_2) & (data.away_team == team_1))) & 
                  (data.date < curr_match.date)]


    count = Counter(df_h2h['winner']) #Now we get no.win and no.draw   
    
    pre_df['h_win_diff'] = count[team_1] - count[team_2]
    pre_df['h_draw'] = count['None']
    
    
    # Get the recent result of each team
    # Team 1
#     team_1 = pre_df['team_1']
    df_10  = data[((data.home_team == team_1) | (data.away_team == team_1)) & 
                 (data.date < curr_match.date)]
    
    length = df_10.shape[0]
    df_10  = df_10.iloc[length-11:length-1,:]
    
    df_10['winner'] = np.where(df_10.home_score > df_10.away_score, df_10.home_team, 
                                     np.where(df_10.home_score < df_10.away_score, df_10.away_team, 'None'))
    
    pre_df['f_goalF_1'] = df_10['home_score'].sum()
    pre_df['f_goalA_1'] = df_10['away_score'].sum()
    
    dict_result_10 = Counter(df_10['winner'])
    pre_df['f_win_1']  = dict_result_10[team_1]
    pre_df['f_draw_1'] = dict_result_10['None']
    
    # Team 2
#     team_2 = pre_df['team_2']
    df_10  = data[((data.home_team == team_2) | (data.away_team == team_2)) & 
                 (data.date < curr_match.date)]
    
    length = df_10.shape[0]
    df_10  = df_10.iloc[length-11:length-1,:]
    
    df_10['winner'] = np.where(df_10.home_score > df_10.away_score, df_10.home_team, 
                                     np.where(df_10.home_score < df_10.away_score, df_10.away_team, 'None'))
    
    pre_df['f_goalF_2'] = df_10['home_score'].sum()
    pre_df['f_goalA_2'] = df_10['away_score'].sum()
    
    dict_result_10 = Counter(df_10['winner'])
    pre_df['f_win_2']  = dict_result_10[team_2]
    pre_df['f_draw_2'] = dict_result_10['None']
    
    pre_df['result'] = np.where(curr_match.winner == team_1, 'win', 
                                     np.where(curr_match.winner == team_2, 'lose', 'draw'))
    
    
    my_df = my_df.append(pre_df)
    
end_time = time.time()
# print(time.time() - start_time)
print('Execution Time : {}'.format(end_time - start_time))
my_df.to_csv('data/data_2005.csv')

Execution Time : 1371.9029848575592


In [124]:
#  Save the file to csv
my_df.to_csv('data/data_2016.csv')

In [118]:
my_df.shape

(11958, 20)

In [120]:
data_2005.shape

(11958, 10)

In [121]:
my_df.head()

,id,date,team_1,team_2,home_team,tournament,h_win_diff,h_draw,f_goalF_1,f_goalF_2,f_goalA_1,f_goalA_2,f_win_1,f_win_2,f_draw_1,f_draw_2,avg_odds_t1_win,avg_odds_draw,avg_odds_t2_win,result
26944,26944.0,2005-01-02,Burma,Singapore,Singapore,AFF Championship,0.0,6.0,17.0,11.0,18.0,18.0,4.0,1.0,2.0,3.0,NaN,NaN,NaN,lose
26945,26945.0,2005-01-03,Indonesia,Malaysia,Malaysia,AFF Championship,0.0,6.0,12.0,15.0,15.0,13.0,4.0,0.0,2.0,3.0,NaN,NaN,NaN,win
26946,26946.0,2005-01-08,Egypt,Uganda,Egypt,Friendly,0.0,6.0,14.0,14.0,6.0,8.0,5.0,4.0,2.0,4.0,NaN,NaN,NaN,win
26947,26947.0,2005-01-08,Indonesia,Singapore,Indonesia,AFF Championship,0.0,6.0,12.0,11.0,15.0,18.0,4.0,1.0,2.0,3.0,NaN,NaN,NaN,lose
26948,26948.0,2005-01-08,French Guyana,Jamaica,Jamaica,CFU Caribbean Cup qualification,0.0,6.0,18.0,15.0,11.0,12.0,6.0,4.0,2.0,3.0,NaN,NaN,NaN,lose


In [115]:
curr_match

date          2018-03-23
home_team        Germany
away_team          Spain
home_score             1
away_score             1
tournament      Friendly
city          Düsseldorf
country          Germany
id                 38792
winner              None
Name: 38792, dtype: object

In [116]:
team_1 = 'Germany'
team_2 = 'Spain'
df_h2h = data[(((data.home_team == team_1) & (data.away_team == team_2)) | 
            ((data.home_team == team_2) & (data.away_team == team_1))) & 
            (data.date < curr_match.date)]
df_h2h

,date,home_team,away_team,home_score,away_score,tournament,city,country,id,winner
1741,1935-05-12,Germany,Spain,1,2,Friendly,Cologne,Germany,1741,Spain
1822,1936-02-23,Spain,Germany,1,2,Friendly,Barcelona,Spain,1822,Germany
2356,1942-04-12,Germany,Spain,1,1,Friendly,Berlin,Germany,2356,None
3355,1952-12-28,Spain,Germany,2,2,Friendly,Madrid,Spain,3355,None
4248,1958-03-19,Germany,Spain,2,0,Friendly,Frankfurt am Main,Germany,4248,Germany
6163,1966-07-20,Germany,Spain,2,1,FIFA World Cup,Birmingham,England,6163,Germany
7298,1970-02-11,Spain,Germany,2,0,Friendly,Seville,Spain,7298,Spain
8755,1973-11-24,Germany,Spain,2,1,Friendly,Stuttgart,Germany,8755,Germany
8820,1974-02-23,Spain,Germany,1,0,Friendly,Barcelona,Spain,8820,Spain
9607,1976-04-24,Spain,Germany,1,1,UEFA Euro qualification,Madrid,Spain,9607,None


In [112]:
list(range(3))

[0, 1, 2]

In [110]:
# How many data we throw
1 - data_2005.shape[0] / data.shape[0]
# we throw 70% of data

0.6926122050280191

In [16]:
# First filter data from 2005
# 2005-01-01 to 2015-07-01
# For example filter data of Germany vs Spain
df1 = data[(data.home_team == 'Spain') & (data.away_team == 'Germany')]
df2 = data[(data.home_team == 'Germany') & (data.away_team == 'Spain')]
df3 = df1.append(df2).sort_values(by=['date'])
df3.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,id
1741,1935-05-12,Germany,Spain,1,2,Friendly,Cologne,Germany,1741
1822,1936-02-23,Spain,Germany,1,2,Friendly,Barcelona,Spain,1822
2356,1942-04-12,Germany,Spain,1,1,Friendly,Berlin,Germany,2356
3355,1952-12-28,Spain,Germany,2,2,Friendly,Madrid,Spain,3355
4248,1958-03-19,Germany,Spain,2,0,Friendly,Frankfurt am Main,Germany,4248


In [62]:
df3

,date,home_team,away_team,home_score,away_score,tournament,city,country,id
1741,1935-05-12,Germany,Spain,1,2,Friendly,Cologne,Germany,1741
1822,1936-02-23,Spain,Germany,1,2,Friendly,Barcelona,Spain,1822
2356,1942-04-12,Germany,Spain,1,1,Friendly,Berlin,Germany,2356
3355,1952-12-28,Spain,Germany,2,2,Friendly,Madrid,Spain,3355
4248,1958-03-19,Germany,Spain,2,0,Friendly,Frankfurt am Main,Germany,4248
6163,1966-07-20,Germany,Spain,2,1,FIFA World Cup,Birmingham,England,6163
7298,1970-02-11,Spain,Germany,2,0,Friendly,Seville,Spain,7298
8755,1973-11-24,Germany,Spain,2,1,Friendly,Stuttgart,Germany,8755
8820,1974-02-23,Spain,Germany,1,0,Friendly,Barcelona,Spain,8820
9607,1976-04-24,Spain,Germany,1,1,UEFA Euro qualification,Madrid,Spain,9607


In [84]:
# define our own datafram
col_names = ['id','date', 'team_1', 'team_2', 'home_team', 'tournament', 'h_win_diff', 'h_draw', 'f_goalF_1', 
             'f_goalF_2','f_goalA_1','f_goalA_2', 'f_win_1', 'f_win_2', 'f_draw_1', 'f_draw_2','result']
my_df = pd.DataFrame(columns = col_names)
my_df

,id,date,team_1,team_2,home_team,tournament,h_win_diff,h_draw,f_goalF_1,f_goalF_2,f_goalA_1,f_goalA_2,f_win_1,f_win_2,f_draw_1,f_draw_2,result


In [18]:
# Now process in df3 to get desired information
# For example FIFA world cup 2010
df_test = df3.copy()
# Add new column = result to df3
df_test['winner'] = np.where(df_test.home_score > df_test.away_score, df_test.home_team, 
                                 np.where(df_test.home_score < df_test.away_score, df_test.away_team, 'None'))
df_test.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,id,winner
1741,1935-05-12,Germany,Spain,1,2,Friendly,Cologne,Germany,1741,Spain
1822,1936-02-23,Spain,Germany,1,2,Friendly,Barcelona,Spain,1822,Germany
2356,1942-04-12,Germany,Spain,1,1,Friendly,Berlin,Germany,2356,None
3355,1952-12-28,Spain,Germany,2,2,Friendly,Madrid,Spain,3355,None
4248,1958-03-19,Germany,Spain,2,0,Friendly,Frankfurt am Main,Germany,4248,Germany


In [ ]:
# Get ten recent matches - sort and take 10
# df_10 = data[data[home_team] == 'Germany' or ]


date          2008-06-29
home_team        Germany
away_team          Spain
home_score             0
away_score             1
tournament     UEFA Euro
city              Vienna
country          Austria
winner             Spain
Name: 30052, dtype: object

In [19]:
curr_match = df_test.iloc[-4, :]
curr_match

date          2008-06-29
home_team        Germany
away_team          Spain
home_score             0
away_score             1
tournament     UEFA Euro
city              Vienna
country          Austria
id                 30052
winner             Spain
Name: 30052, dtype: object

In [20]:
df_match = df_test[df_test['date'] < curr_match['date'] ]
df_match.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,id,winner
1741,1935-05-12,Germany,Spain,1,2,Friendly,Cologne,Germany,1741,Spain
1822,1936-02-23,Spain,Germany,1,2,Friendly,Barcelona,Spain,1822,Germany
2356,1942-04-12,Germany,Spain,1,1,Friendly,Berlin,Germany,2356,None
3355,1952-12-28,Spain,Germany,2,2,Friendly,Madrid,Spain,3355,None
4248,1958-03-19,Germany,Spain,2,0,Friendly,Frankfurt am Main,Germany,4248,Germany


In [31]:
%config IPCompleter.greedy=True

In [27]:
# Now we do history analysis here
from collections import Counter
count = Counter(df_match['winner']) #Now we get no.win and no.draw
count

Counter({'Germany': 8, 'None': 6, 'Spain': 5})

In [57]:
count['Germany']

8

In [58]:
# Now fill in info in our table
# col_names = ['id','date', 'team_1', 'team_2', 'home_team', 'tournament', 'h_win_diff', 'h_draw', 'f_num_goal_1', 
#              'f_num_goal_2', 'f_win_1', 'f_win_2', 'f_draw_1', 'f_draw_2']
# pre_df = pd.DataFrame(columns = col_names)
# my_df

pre_df = curr_match[['id','date','tournament']].copy()
# 
pre_df['team_1'] =  curr_match['home_team'] if curr_match['home_team'] < curr_match['away_team'] else curr_match['away_team'] 
pre_df['team_2'] =  curr_match['away_team'] if curr_match['home_team'] < curr_match['away_team'] else curr_match['away_team'] 
pre_df['home_team'] = curr_match['home_team']

pre_df['h_win_diff'] = count[pre_df['team_1']] - count[pre_df['team_2']]
pre_df['h_draw'] = count['None']


In [105]:
my_df

,id,date,team_1,team_2,home_team,tournament,h_win_diff,h_draw,f_goalF_1,f_goalF_2,f_goalA_1,f_goalA_2,f_win_1,f_win_2,f_draw_1,f_draw_2,result
30052,30052.0,2008-06-29,Germany,Spain,Germany,UEFA Euro,3.0,6.0,23.0,18.0,12.0,19.0,7.0,7.0,3.0,3.0,lose
31737,31737.0,2010-07-07,Germany,Spain,Germany,FIFA World Cup,2.0,6.0,23.0,18.0,12.0,19.0,7.0,7.0,3.0,3.0,lose
36013,36013.0,2014-11-18,Germany,Spain,Spain,Friendly,1.0,6.0,23.0,18.0,12.0,19.0,7.0,7.0,3.0,3.0,win
38792,38792.0,2018-03-23,Germany,Spain,Germany,Friendly,2.0,6.0,23.0,18.0,12.0,19.0,7.0,7.0,3.0,3.0,draw


In [103]:
curr_match

date          2018-03-23
home_team        Germany
away_team          Spain
home_score             1
away_score             1
tournament      Friendly
city          Düsseldorf
country          Germany
id                 38792
winner              None
Name: 38792, dtype: object

In [97]:
df_test

,date,home_team,away_team,home_score,away_score,tournament,city,country,id,winner
1741,1935-05-12,Germany,Spain,1,2,Friendly,Cologne,Germany,1741,Spain
1822,1936-02-23,Spain,Germany,1,2,Friendly,Barcelona,Spain,1822,Germany
2356,1942-04-12,Germany,Spain,1,1,Friendly,Berlin,Germany,2356,None
3355,1952-12-28,Spain,Germany,2,2,Friendly,Madrid,Spain,3355,None
4248,1958-03-19,Germany,Spain,2,0,Friendly,Frankfurt am Main,Germany,4248,Germany
6163,1966-07-20,Germany,Spain,2,1,FIFA World Cup,Birmingham,England,6163,Germany
7298,1970-02-11,Spain,Germany,2,0,Friendly,Seville,Spain,7298,Spain
8755,1973-11-24,Germany,Spain,2,1,Friendly,Stuttgart,Germany,8755,Germany
8820,1974-02-23,Spain,Germany,1,0,Friendly,Barcelona,Spain,8820,Spain
9607,1976-04-24,Spain,Germany,1,1,UEFA Euro qualification,Madrid,Spain,9607,None


In [104]:
# Assemble
# Before this loop, I need to add winner here
import time

# Drop all data
my_df = my_df[0:0]

# need to get df_test here

start_time = time.time()
# Test history match stat
for i in range(-4,0):
    print(i)
    curr_match = df_test.iloc[i,:]
    df_match = df_test[df_test['date'] < curr_match['date'] ]
    count = Counter(df_match['winner']) #Now we get no.win and no.draw
    
    pre_df = curr_match[['id','date','tournament']].copy()
    
#     	id	date	team_1	team_2	home_team	tournament	h_win_diff	h_draw	f_goalF_1	f_goalF_2	f_goalA_1	f_goalA_2	f_win_1	f_win_2	f_draw_1	f_draw_2	result
    # Get the head-to-head result
    pre_df['team_1'] =  curr_match['home_team'] if curr_match['home_team'] < curr_match['away_team'] else curr_match['away_team'] 
    pre_df['team_2'] =  curr_match['away_team'] if curr_match['home_team'] < curr_match['away_team'] else curr_match['home_team'] 
    pre_df['home_team'] = curr_match['home_team']

    pre_df['h_win_diff'] = count[pre_df['team_1']] - count[pre_df['team_2']]
    pre_df['h_draw'] = count['None']
    
    # Get the recent result of each team
    # Team 1
    team_1 = pre_df['team_1']
    df_10  = data[(data.home_team == team_1) | (data.away_team == team_1)]
    
    length = df_10.shape[0]
    df_10  = df_10.iloc[length-11:length-1,:]
    
    df_10['winner'] = np.where(df_10.home_score > df_10.away_score, df_10.home_team, 
                                     np.where(df_10.home_score < df_10.away_score, df_10.away_team, 'None'))
    
    pre_df['f_goalF_1'] = df_10['home_score'].sum()
    pre_df['f_goalA_1'] = df_10['away_score'].sum()
    
    dict_result_10 = Counter(df_10['winner'])
    pre_df['f_win_1']  = dict_result_10[team_1]
    pre_df['f_draw_1'] = dict_result_10['None']
    
    # Team 2
    team_2 = pre_df['team_2']
    df_10  = data[(data.home_team == team_2) | (data.away_team == team_2)]
    
    length = df_10.shape[0]
    df_10  = df_10.iloc[length-11:length-1,:]
    
    df_10['winner'] = np.where(df_10.home_score > df_10.away_score, df_10.home_team, 
                                     np.where(df_10.home_score < df_10.away_score, df_10.away_team, 'None'))
    
    pre_df['f_goalF_2'] = df_10['home_score'].sum()
    pre_df['f_goalA_2'] = df_10['away_score'].sum()
    
    dict_result_10 = Counter(df_10['winner'])
    pre_df['f_win_2']  = dict_result_10[team_2]
    pre_df['f_draw_2'] = dict_result_10['None']
    
    pre_df['result'] = np.where(curr_match.winner == team_1, 'win', 
                                     np.where(curr_match.winner == team_2, 'lose', 'draw'))
    
    
    my_df = my_df.append(pre_df)
    
print(time.time() - start_time)

-4
-3
-2
-1
2.9774169921875


In [95]:
my_df = my_df[0:0]
my_df

,id,date,team_1,team_2,home_team,tournament,h_win_diff,h_draw,f_goalF_1,f_goalF_2,f_goalA_1,f_goalA_2,f_win_1,f_win_2,f_draw_1,f_draw_2,result


In [89]:
data.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,id
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,0
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,1
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,2
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,3
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,4


In [93]:
#  Ok now move to the next step: retrieve the recent performance 
team_1 = curr_match['home_team']

df_10  = data[(data.home_team == team_1) | (data.away_team == team_1)]
length = df_10.shape[0]
df_10  = df_10.iloc[length-11:length-1,:]
df_10['winner'] = np.where(df_10.home_score > df_10.away_score, df_10.home_team, 
                                 np.where(df_10.home_score < df_10.away_score, df_10.away_team, 'None'))
df_10

# df2 = data[(data.home_team == 'Germany') & (data.away_team == 'Spain')]
# df3 = df1.append(df2).sort_values(by=['date'])
# df_10.iloc[:]

,date,home_team,away_team,home_score,away_score,tournament,city,country,id,winner
38256,2017-06-25,Germany,Cameroon,3,1,Confederations Cup,Sochi,Russia,38256,Germany
38268,2017-06-29,Germany,Mexico,4,1,Confederations Cup,Sochi,Russia,38268,Germany
38279,2017-07-02,Chile,Germany,0,1,Confederations Cup,St Petersburg,Russia,38279,Germany
38378,2017-09-01,Czech Republic,Germany,1,2,FIFA World Cup qualification,Prague,Czech Republic,38378,Germany
38415,2017-09-04,Germany,Norway,6,0,FIFA World Cup qualification,Stuttgart,Germany,38415,Germany
38476,2017-10-05,Northern Ireland,Germany,1,3,FIFA World Cup qualification,Belfast,Northern Ireland,38476,Germany
38528,2017-10-08,Germany,Azerbaijan,5,1,FIFA World Cup qualification,Kaiserslautern,Germany,38528,Germany
38609,2017-11-10,England,Germany,0,0,Friendly,London,England,38609,None
38678,2017-11-14,Germany,France,2,2,Friendly,Cologne,Germany,38678,None
38792,2018-03-23,Germany,Spain,1,1,Friendly,Düsseldorf,Germany,38792,None


In [94]:
df_10['home_score'].sum()
df_10['away_score'].sum()

23

In [96]:
counter = Counter(df_10['winner'])

In [102]:
counter

Counter({'Germany': 7, 'None': 3})

In [101]:
counter['Germany']

7

In [82]:
# df_10.shape
length = df_10.shape[0]

In [92]:
df_10.iloc[length-11:length-1,:]

,date,home_team,away_team,home_score,away_score,tournament,city,country,id,winner
38256,2017-06-25,Germany,Cameroon,3,1,Confederations Cup,Sochi,Russia,38256,Germany
38268,2017-06-29,Germany,Mexico,4,1,Confederations Cup,Sochi,Russia,38268,Germany
38279,2017-07-02,Chile,Germany,0,1,Confederations Cup,St Petersburg,Russia,38279,Germany
38378,2017-09-01,Czech Republic,Germany,1,2,FIFA World Cup qualification,Prague,Czech Republic,38378,Germany
38415,2017-09-04,Germany,Norway,6,0,FIFA World Cup qualification,Stuttgart,Germany,38415,Germany
38476,2017-10-05,Northern Ireland,Germany,1,3,FIFA World Cup qualification,Belfast,Northern Ireland,38476,Germany
38528,2017-10-08,Germany,Azerbaijan,5,1,FIFA World Cup qualification,Kaiserslautern,Germany,38528,Germany
38609,2017-11-10,England,Germany,0,0,Friendly,London,England,38609,None
38678,2017-11-14,Germany,France,2,2,Friendly,Cologne,Germany,38678,None
38792,2018-03-23,Germany,Spain,1,1,Friendly,Düsseldorf,Germany,38792,None


In [76]:
curr_match

date          2018-03-23
home_team        Germany
away_team          Spain
home_score             1
away_score             1
tournament      Friendly
city          Düsseldorf
country          Germany
id                 38792
winner              None
Name: 38792, dtype: object

True